In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

def optimize_datatypes(data):
    # Downcast numerical columns
    for col in data.select_dtypes(include='int'):
        data[col] = pd.to_numeric(data[col], downcast='integer')
    for col in data.select_dtypes(include='float'):
        data[col] = pd.to_numeric(data[col], downcast='float')
    
    # Convert categorical columns to category dtype
    if 'Hypertension' in data.columns:
        data['Hypertension'] = data['Hypertension'].astype('category')
    
    return data

def mainBareBones():
    # Load the dataset
    file_path = "diabetes_dataset.csv"  # Replace with your dataset's file path
    print("Loading dataset...")
    try:
        data = pd.read_csv(file_path)
        print(f"Dataset loaded successfully! Shape: {data.shape}")
    except Exception as e:
        print(f"Error loading dataset: {e}")
        exit()

    # Data Cleaning and Preprocessing
    print("\nCleaning and preprocessing data...")
    print(f"Missing values before cleaning: {data.isnull().sum().sum()}")
    data.fillna(data.mean(numeric_only=True), inplace=True)  # Replace NaNs in numeric columns
    print(f"Missing values after cleaning: {data.isnull().sum().sum()}")

    # Optimize data types
    print("\nOptimizing data types...")
    data = optimize_datatypes(data)

    # Encode categorical variables (if applicable)
    if 'Hypertension' in data.columns:
        data['Hypertension'] = data['Hypertension'].cat.codes

    # Separate features (X) and target (y)
    print("\nSeparating features and target variable...")
    X = data.drop('Outcome', axis=1)
    y = data['Outcome']
    print(f"Features shape: {X.shape}, Target shape: {y.shape}")

    # Split the data into training and testing sets
    print("\nSplitting data into training and testing sets...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Training set shape: {X_train.shape}, Testing set shape: {X_test.shape}")

    # Scale features for better performance
    print("\nScaling features...")
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Initialize and train the logistic regression model
    print("\nTraining the logistic regression model...")
    model = LogisticRegression(random_state=42, max_iter=1000)
    model.fit(X_train, y_train)
    print("Model training completed!")

    # Make predictions
    print("\nMaking predictions on the test set...")
    y_pred = model.predict(X_test)

    # Evaluate the model
    print("\nEvaluating the model...")
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

if __name__ == "__main__":
    mainBareBones()

Loading dataset...
Dataset loaded successfully! Shape: (9538, 17)

Cleaning and preprocessing data...
Missing values before cleaning: 0
Missing values after cleaning: 0

Optimizing data types...

Separating features and target variable...
Features shape: (9538, 16), Target shape: (9538,)

Splitting data into training and testing sets...
Training set shape: (7630, 16), Testing set shape: (1908, 16)

Scaling features...

Training the logistic regression model...
Model training completed!

Making predictions on the test set...

Evaluating the model...
Accuracy: 99.32%

Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1275
           1       1.00      0.98      0.99       633

    accuracy                           0.99      1908
   macro avg       0.99      0.99      0.99      1908
weighted avg       0.99      0.99      0.99      1908



In [2]:
import pandas as pd
import cupy as cp
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

def optimize_datatypes(data):
    # Downcast numerical columns to reduce memory usage
    for col in data.select_dtypes(include=['int']):
        data[col] = pd.to_numeric(data[col], downcast='integer')
    for col in data.select_dtypes(include=['float']):
        data[col] = pd.to_numeric(data[col], downcast='float')
    
    return data

def mainCuPY():
    # Load the dataset
    file_path = "diabetes_dataset.csv"  # Replace with your file path
    print("Loading dataset...")
    try:
        data = pd.read_csv(file_path)
        print(f"Dataset loaded successfully! Shape: {data.shape}")
    except Exception as e:
        print(f"Error loading dataset: {e}")
        exit()

    # Data Cleaning and Preprocessing
    print("\nCleaning and preprocessing data...")
    missing_values_before = data.isnull().sum().sum()
    print(f"Missing values before cleaning: {missing_values_before}")
    data.fillna(data.mean(numeric_only=True), inplace=True)  # Fill NaNs for numeric columns
    missing_values_after = data.isnull().sum().sum()
    print(f"Missing values after cleaning: {missing_values_after}")

    # Optimize datatypes
    print("\nOptimizing data types...")
    data = optimize_datatypes(data)

    # Convert categorical columns (e.g., Hypertension) to integers if necessary
    if 'Hypertension' in data.columns:
        data['Hypertension'] = data['Hypertension'].astype(int)

    # Separate features (X) and target (y)
    print("\nSeparating features and target variable...")
    X = data.drop('Outcome', axis=1).values
    y = data['Outcome'].values
    print(f"Features shape: {X.shape}, Target shape: {y.shape}")

    # Split the data into training and testing sets
    print("\nSplitting data into training and testing sets...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Training set shape: {X_train.shape}, Testing set shape: {X_test.shape}")

    # Convert to CuPy arrays for GPU acceleration
    print("\nTransferring data to GPU using CuPy...")
    X_train_cp = cp.asarray(X_train, dtype=cp.float32)
    X_test_cp = cp.asarray(X_test, dtype=cp.float32)
    y_train_cp = cp.asarray(y_train, dtype=cp.int32)
    y_test_cp = cp.asarray(y_test, dtype=cp.int32)

    # Standardize features
    print("\nScaling features...")
    scaler = StandardScaler()
    X_train_scaled = cp.asarray(scaler.fit_transform(cp.asnumpy(X_train_cp)))
    X_test_scaled = cp.asarray(scaler.transform(cp.asnumpy(X_test_cp)))

    # Train the logistic regression model
    print("\nTraining the logistic regression model...")
    model = LogisticRegression(random_state=42, max_iter=1000)
    model.fit(cp.asnumpy(X_train_scaled), cp.asnumpy(y_train_cp))
    print("Model training completed!")

    # Make predictions
    print("\nMaking predictions on the test set...")
    y_pred = model.predict(cp.asnumpy(X_test_scaled))

    # Evaluate the model
    print("\nEvaluating the model...")
    accuracy = accuracy_score(cp.asnumpy(y_test_cp), y_pred)
    print(f"Accuracy (CuPy Optimized): {accuracy * 100:.2f}%")
    print("\nClassification Report:")
    print(classification_report(cp.asnumpy(y_test_cp), y_pred))

if __name__ == "__main__":
    mainCuPY()

Loading dataset...
Dataset loaded successfully! Shape: (9538, 17)

Cleaning and preprocessing data...
Missing values before cleaning: 0
Missing values after cleaning: 0

Optimizing data types...

Separating features and target variable...
Features shape: (9538, 16), Target shape: (9538,)

Splitting data into training and testing sets...
Training set shape: (7630, 16), Testing set shape: (1908, 16)

Transferring data to GPU using CuPy...

Scaling features...

Training the logistic regression model...
Model training completed!

Making predictions on the test set...

Evaluating the model...
Accuracy (CuPy Optimized): 99.32%

Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1275
           1       1.00      0.98      0.99       633

    accuracy                           0.99      1908
   macro avg       0.99      0.99      0.99      1908
weighted avg       0.99      0.99      0.99      1908



In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearnex import patch_sklearn 
patch_sklearn() # Apply Intel optimizations
def optimize_datatypes(data):
    # Downcast numerical columns
    for col in data.select_dtypes(include='int'):
        data[col] = pd.to_numeric(data[col], downcast='integer')
    for col in data.select_dtypes(include='float'):
        data[col] = pd.to_numeric(data[col], downcast='float')
    
    # Convert categorical columns to category dtype
    if 'Hypertension' in data.columns:
        data['Hypertension'] = data['Hypertension'].astype('category')
    
    return data

def mainIntelex():
    # Load the dataset
    file_path = "diabetes_dataset.csv"  # Replace with your dataset's file path
    print("Loading dataset...")
    try:
        data = pd.read_csv(file_path)
        print(f"Dataset loaded successfully! Shape: {data.shape}")
    except Exception as e:
        print(f"Error loading dataset: {e}")
        exit()

    # Data Cleaning and Preprocessing
    print("\nCleaning and preprocessing data...")
    print(f"Missing values before cleaning: {data.isnull().sum().sum()}")
    data.fillna(data.mean(numeric_only=True), inplace=True)  # Replace NaNs in numeric columns
    print(f"Missing values after cleaning: {data.isnull().sum().sum()}")

    # Optimize data types
    print("\nOptimizing data types...")
    data = optimize_datatypes(data)

    # Encode categorical variables (if applicable)
    if 'Hypertension' in data.columns:
        data['Hypertension'] = data['Hypertension'].cat.codes

    # Separate features (X) and target (y)
    print("\nSeparating features and target variable...")
    X = data.drop('Outcome', axis=1)
    y = data['Outcome']
    print(f"Features shape: {X.shape}, Target shape: {y.shape}")

    # Split the data into training and testing sets
    print("\nSplitting data into training and testing sets...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Training set shape: {X_train.shape}, Testing set shape: {X_test.shape}")

    # Scale features for better performance
    print("\nScaling features...")
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Initialize and train the logistic regression model
    print("\nTraining the logistic regression model...")
    model = LogisticRegression(random_state=42, max_iter=1000)
    model.fit(X_train, y_train)
    print("Model training completed!")

    # Make predictions
    print("\nMaking predictions on the test set...")
    y_pred = model.predict(X_test)

    # Evaluate the model
    print("\nEvaluating the model...")
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

if __name__ == "__main__":
    mainIntelex()

Loading dataset...
Dataset loaded successfully! Shape: (9538, 17)

Cleaning and preprocessing data...
Missing values before cleaning: 0
Missing values after cleaning: 0

Optimizing data types...

Separating features and target variable...
Features shape: (9538, 16), Target shape: (9538,)

Splitting data into training and testing sets...
Training set shape: (7630, 16), Testing set shape: (1908, 16)

Scaling features...

Training the logistic regression model...
Model training completed!

Making predictions on the test set...

Evaluating the model...
Accuracy: 99.32%

Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1275
           1       1.00      0.98      0.99       633

    accuracy                           0.99      1908
   macro avg       0.99      0.99      0.99      1908
weighted avg       0.99      0.99      0.99      1908



Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
